In [104]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re

driver = webdriver.Chrome(executable_path=r"C:\Users\Mark\Anaconda3\selenium\webdriver\chrome\chromedriver.exe")

In [105]:
driver.get('https://www.beeradvocate.com/lists/popular/')

In [106]:
beer_html = driver.page_source
beer_soup = BeautifulSoup(beer_html, 'lxml')

In [107]:
# name and url
beer_list = []
for i in beer_soup.find_all('td', {'valign': 'middle'})[4:]:
    beer_dict = {}
    beer_dict['name'] = i.find('a').text
    beer_dict['url'] = i.find('a').attrs['href']
    beer_list.append(beer_dict)

df_beer = pd.DataFrame(beer_list)
df_beer.index += 1

# score
beer_score = []
for i in beer_soup.find_all('td', {'class': 'hr_bottom_light', 'align':'left', 'valign':'top'}):
    beer_score.append(i.text)
df_beer['score'] = beer_score

# ratings count
beer_ratings = []
for i in beer_soup.find_all('td', {'class': 'hr_bottom_light', 'align':'right'}):
    beer_ratings.append(i.text)
df_beer['ratings'] = beer_ratings

df_beer.head(2)

,name,url,score,ratings
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948"
2,90 Minute IPA,/beer/profile/64/2093/,4.27,"15,782"


### Info and Review on the beer

In [108]:
beer_resr = requests.get('https://www.beeradvocate.com/beer/profile/1199/11757/')
beer_soupr = BeautifulSoup(beer_resr.content, 'lxml')

In [109]:
brewery_list = []
for i in beer_soupr.find_all('div', {'id':'info_box'}):
    brewery_list.append(i.find('a').text)

brewery_list

['Founders Brewing Company']

In [110]:
beer_text = beer_soupr.find('div', {'id':'info_box'}).text

In [155]:
abv_grabber = lambda x: re.search('\(ABV\): (\d+\.\d+)%', x).group(1)
notes_grabber = lambda x: re.search('Description:[\s\S]+ IBU', x).group(0)
ibu_grabber = lambda x: re.search('(\d*) IBU', x).group(1)

def notes_cleaner(string):
    x = notes_grabber(string)
    x = re.sub('\s', ' ', x)
    x = re.sub('\d* IBU', '', x)
    x = re.sub('Availability:', '', x)
    x = re.sub('Description:', '', x)
    return x

In [149]:
beer_text

'\n\nBEER INFO\n\nBrewed by:\n\nFounders Brewing Company\nMichigan, United Statesfoundersbrewing.com \nStyle: American Double / Imperial Stout\n\nAlcohol by volume (ABV): 8.30%\n\t\t\nAvailability: Rotating\n\t\t\nNotes / Commercial Description:\n\n\t\tAvailability: Sep - Dec\n\r\nBrewed with flaked oats, bitter and sweetened imported chocolates, Sumatra and Kona coffee.\n\r\n60 IBUAdded by marburg on 07-26-2003 '

In [150]:
abv_grabber(beer_text)

'8.30'

In [151]:
notes_cleaner(notes_grabber(beer_text))

'     Sep - Dec   Brewed with flaked oats, bitter and sweetened imported chocolates, Sumatra and Kona coffee.   '

In [156]:
ibu_grabber(beer_text)

'60'

In [ ]:
# grab IBU
\d* IBU

#### Test on a second beer

In [51]:
# beer_resr2 = requests.get('https://www.beeradvocate.com/beer/profile/64/2093/')
# beer_soupr2 = BeautifulSoup(beer_resr2.content, 'lxml')

In [123]:
# for i in beer_soupr2.find_all('div', {'id':'info_box'}):
#     print(i.find('a').text)

In [124]:
# beer_text2 = beer_soupr2.find('div', {'id':'info_box'}).text

In [125]:
# beer_text2

In [126]:
# abv_grabber(beer_text2)

In [127]:
# notes_cleaner(notes_grabber(beer_text2))

### Beer Reviews

In [115]:
driver.get('https://www.beeradvocate.com/beer/profile/64/2093/')

In [116]:
reviews_html = driver.page_source
reviews_soup = BeautifulSoup(reviews_html, 'lxml')

In [ ]:
# User score -- span | class = BAscore_norm
# Look/Smell/Feel/Taste... -- span | class = muted
# Characters -- span | class = muted <-- not using
# username -- span | class = muted  &  a href | class = username

# the container for all the revies
# div id= rating_fullview_container

In [128]:
# where reviews are located
# reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})

In [129]:
# for i in reviews_soup.find_all('div', {'id':'rating_fullview_content_2'}):
#     print(i)

In [135]:
reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})

[<div id="rating_fullview_content_2"><span class="BAscore_norm">4.29</span><span class="rAvg_norm">/5</span>  rDev <span style="color:#006600;">+0.5%</span><br/><span class="muted">look: 4.25 | smell: 4 | taste: 4.5 | feel: 4.25 |  overall: 4.25</span><br/><br/><div><span class="muted"><a class="username" href="/community/members/derdtheterd.1080390/">derdtheterd</a>, <a href="/beer/profile/64/2093/?ba=derdtheterd#review">Yesterday at 04:56 AM</a></span></div></div>,
 <div id="rating_fullview_content_2"><span class="BAscore_norm">4</span><span class="rAvg_norm">/5</span>  rDev <span style="color:#990000;">-6.3%</span><br/><span class="muted">look: 4 | smell: 4 | taste: 4 | feel: 4 |  overall: 4</span><br/><br/><div><span class="muted"><a class="username" href="/community/members/jasoncpace.680066/">Jasoncpace</a>, <a href="/beer/profile/64/2093/?ba=Jasoncpace#review">Sunday at 11:14 PM</a></span></div></div>,
 <div id="rating_fullview_content_2"><span class="BAscore_norm">4.01</span><s

In [134]:
reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})[5].text

"4.56/5\xa0\xa0rDev +6.8%look: 4.25 | smell: 4.5 | taste: 4.75 | feel: 4.25 |  overall: 4.5One of the best IPAs I've ever had. Nothing was in excess. Well-rounded flavors, tastes, smells, etc. Love it.110 charactersMiniPigLover, Friday at 04:06 PM"

In [130]:
review_grabber = lambda x: re.search('overall:\s\d*\.\d(.*\d* characters)', x).group(1)

In [131]:
abc = review_grabber(reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})[5].text)

In [132]:
re.sub('\d* characters', '', abc)

"One of the best IPAs I've ever had. Nothing was in excess. Well-rounded flavors, tastes, smells, etc. Love it."

In [143]:
review_grabber(reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})[5].text)

"One of the best IPAs I've ever had. Nothing was in excess. Well-rounded flavors, tastes, smells, etc. Love it.110 characters"

In [ ]:
# put into a try/except loop

In [147]:
for i in range(1,26):
    try:
        print(review_grabber(reviews_soup.find_all('div', {'id':'rating_fullview_content_2'})[i].text))
    except:
        print("No Review")

No Review
No Review
No Review
No Review
One of the best IPAs I've ever had. Nothing was in excess. Well-rounded flavors, tastes, smells, etc. Love it.110 characters
No Review
No Review
No Review
No Review
5Great beer with a well balance body. Look was sharp amber and clear. Smell was aromatic with a bright and complex hop profile. I personally loved the taste and the feel complimented the body well. Nice malty tone that helped keep the high gravity manageable on the palate. Another great beer from Dogfish. 305 characters
No Review
For me it all started on a snowy day in 2015, I was in a small college town of Monmouth Oregon. Beer was just recently introduced to me by a family friend. Not only did he tell me it would change my life, but it would affect how I did things from here on out. It did change my life, and it started with the 90 minute IPA by Dogfish Head. I was working at a winery at the time, but it was so snowy outside (oregon terms) because of this work was cancelled. I was s

In [159]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Mark\Anaconda3\selenium\webdriver\chrome\chromedriver.exe")
driver.get('https://www.beeradvocate.com/beer/profile/1199/11757/?view=beer&sort=&start=0')
beer_page = driver.page_source
beer_page_soup = BeautifulSoup(beer_page, 'lxml')

In [173]:
review_grabber = lambda x: re.search('overall:\s\d\.\d*(.*[\s\S]*\d characters)', x).group(1)

def review_cleaner(string):
    x = review_grabber(string)
    x = re.sub('\d* characters', '', x)
    x = re.sub('\n', ' ', x)
    return x

In [174]:
# review = review_grabber(beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})[7].text)
review_cleaner(beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})[7].text)

"poured from the tap of my kegerator into a porter/stout glass  A - 2 fingers medium/dark tan over a body of pitch black; laces the glass in nice rings  S - lots of chocolate, coffee, roasted malts, suggestion of dark fruits  T - chocolate, rich black coffee, hint of dark fruitiness  M - thick full body, medium carbonation  O - I'm glad I bought this by the keg!\xa0"

#### Grabing all the other stuffs

In [121]:
user_score = []
score_breakdown = []
user_names = []

for i in reviews_soup.find_all('div', {'id':'rating_fullview_content_2'}):
    user_score.append(i.find('span', {'class': 'BAscore_norm'}).text)
    score_breakdown.append(i.find('span', {'class': 'muted'}).text)
    user_names.append(i.find('a', {'class':'username'}).text)